In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('jeju.csv',encoding='cp949')
df.head(1)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20


In [16]:
#1. 주소 정보를 이용해서 위도, 경도 값을 저장('위치'컬럼에 (위도, 경도) 저장)
# 주소를 찾아서 저장하는 함수를 작성하여 처리
# 함수 이름은 geo_coder(query) 리턴값: (위도, 경도) 만일 못 찾았다면 '결과 없음'으로 리턴
# 함수 사용, apply()를 이용해 데이터 프레임에 적용

In [17]:
query = '제주특별자치도 제주시 중앙로25길 17(일도일동)'
def geo_coder(query):
    import requests # 파이썬으로 HTTP 호출하는 프로그램을 작성할 때 가장 많이 사용되는 라이브러리
    REST_API_KEY = '8f7c4764339397218bd8c18f6d678579'
    query = query.split('(')[0]
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?category_group_code=PO3&query={query}'
    headers = {'Authorization':f'KakaoAK {REST_API_KEY}'}
    res = requests.get(url,headers=headers) # url이라는 주소로 get 요청(requess)를 보냈고 
    jsondata = res.json()
    # print(query)
    # print(jsondata)
    try:
        return jsondata['documents'][0]['y'], jsondata['documents'][0]['x']
    except:
        return '결과없음'

In [18]:
df['위치'] = df['주소'].apply(geo_coder)

In [37]:
df.head()

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,위치
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,"(33.5150737361162, 126.526342790758)"
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28 (일도이동),인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20,결과없음
2,제주시,이도1동,064-728-1530,제주특별자치도 제주시 중앙로25길 17 (이도1동),인구 7418명 / 면적 0.79㎢ / 16통 97개반,지상 2층 / 2층면적 205㎡,개소일 2000-12-28,2019-06-20,"(33.5069402044009, 126.526976882749)"
3,제주시,이도2동,064-755-2021,제주특별자치도 제주시 오복3길 8 (이도이동),인구 41428명 / 면적 5.4㎢ / 49통 323반,지하 1층·지상 3층 / 연면적 1223.52㎡,총면적 388.9㎡ / 개소일 2000-10-05,2019-06-20,"(33.4970469392912, 126.535290947586)"
4,제주시,삼도1동,064-728-4532,제주특별자치도 제주시 전농로 40 (삼도일동),인구 14281명 / 면적0.87㎢ / 18통122반,지하 1층·지상3층 / 연면적 2070.37㎡,총면적 456.55㎡,2019-06-20,"(33.5041154034239, 126.517362400655)"


In [20]:
#2. '위치' 칼럼의 값을 Marker 위치값으로, tooltip으로 읍면동을 사용하여 지도에 표시(클러스터 이용)
# 함수 사용, apply()를 이용해 데이터 프레임에 적용

In [27]:
import folium
from folium.plugins import MarkerCluster

In [22]:
def display(loc,map):
   # print(loc[0],loc[1])
    if loc[0] != '결과없음':
        folium.Marker(loc[0],tooltip=loc[1]).add_to(map)

In [31]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
map_c = MarkerCluster().add_to(map)
df[['위치','읍면동']].apply(display,map = map_c,axis=1)
map 

In [30]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
map_c = MarkerCluster().add_to(map)
for index, row in df.iterrows():
    # print(index)
    # print(row['읍면동'],row['위치'])
    if row['위치'] != '결과없음':
        folium.Marker(row['위치'],tooltip=row['읍면동']).add_to(map_c)
map

In [13]:
#3. '일반현황' 칼럼의 인구수만 분리하여 '인구수'라는 int형 칼럼 생성

In [33]:
df['일반현황'].str.split('/').str.get(0).str.split('(').str.get(0).str.split().str.get(-1).str.replace('명','').astype(int)

0      3146
1     38400
2      7418
3     41428
4     14281
5      9431
6      8368
7     16701
8     10850
9     25559
10    25000
11     3046
12    28741
13    13729
14    39897
15    56223
16    17606
17     4104
18     2837
19    19925
20    26539
21    15080
22    20804
23     8745
24     2820
25     1718
26    16610
27    19151
28    13866
29    11867
30    11167
31     5207
32     2444
33     4003
34     3798
35     5475
36     4980
37    22225
38    10697
39    10344
40    11310
41    10684
42     3813
Name: 일반현황, dtype: int32

In [ ]:
import re

df['인구수'] = df['일반현황'].apply(lambda x:int(re.findall('[0-9]{4,}',x)[-1]))

In [32]:
df.head(2)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,위치
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,"(33.5150737361162, 126.526342790758)"
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28 (일도이동),인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20,결과없음
